In [ ]:
!pip install nltk
!pip install tflearn

In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer

In [3]:
stemmer = LancasterStemmer()

In [4]:
import numpy
import tflearn
import tensorflow
import random
import json

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [5]:
with open("intents.json") as intents:
    data = json.load(intents)
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip cookies for $2!', 'Cookies ar

In [6]:
data['intents']

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Have a Good day'],
  'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
  'context_set': ''},
 {'tag': 'age',
  'patterns': ['how old',
   'how old is tim',
   'what is your age',
   'how old are you',
   'age?'],
  'responses': ['I am 18 years old!', '18 years young!'],
  'context_set': ''},
 {'tag': 'name',
  'patterns': ['what is your name',
   'what should I call you',
   'whats your name?'],
  'responses': ['You can call me Tim.',
   "I'm Tim!",
   "I'm Tim aka Tech With Tim."],
  'context_set': ''},
 {'tag': 'shop',
  'patterns': ['Id like to buy something',
   'whats on the menu',
   'what do you reccommend?',
   'could i get som

In [7]:
#all words
words = []
#the possible labels
labels = []
#will hold the list of patterns
docs_x = []
#will hold the label for each pattern
docs_y = []

#loop through the intent dictionary
# for intent in data['intents']:
#     #loop through each pattern in each patterns list
#     for pattern in intent['patterns']:
#         #tokenize basically separates each word in a pattern and puts the words into a list
#         wrds = nltk.word_tokenize(pattern)
#         print(wrds)

In [8]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        #add all the wrds into words
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
print(words)

#cuts off the last letter of the word
print(stemmer.stem(words[3]))
len(words)

['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'day', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', '?', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', '?', 'Id', 'like', 'to', 'buy', 'something', 'whats', 'on', 'the', 'menu', 'what', 'do', 'you', 'reccommend', '?', 'could', 'i', 'get', 'something', 'to', 'eat', 'when', 'are', 'you', 'guys', 'open', 'what', 'are', 'your', 'hours', 'hours', 'of', 'operation']
you


86

In [9]:
#removes the ends of the words
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
print(words)
print(len(words))

['hi', 'how', 'ar', 'you', 'is', 'anyon', 'ther', 'hello', 'good', 'day', 'what', 'up', 'cya', 'see', 'you', 'lat', 'goodby', 'i', 'am', 'leav', 'hav', 'a', 'good', 'day', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'yo', 'ag', 'how', 'old', 'ar', 'you', 'ag', 'what', 'is', 'yo', 'nam', 'what', 'should', 'i', 'cal', 'you', 'what', 'yo', 'nam', 'id', 'lik', 'to', 'buy', 'someth', 'what', 'on', 'the', 'menu', 'what', 'do', 'you', 'reccommend', 'could', 'i', 'get', 'someth', 'to', 'eat', 'when', 'ar', 'you', 'guy', 'op', 'what', 'ar', 'yo', 'hour', 'hour', 'of', 'op']
82


In [10]:
#remove duplicates
words = sorted(list(set(words)))
len(words)

46

In [11]:
labels = sorted(labels)
print(len(labels))

6


In [12]:
#One hot encoding
#the mapping is [the, she, he, him, they, was, a, guy, person]
#one-hot representation -> [0, 0, 1, 0, 0, 1, 1, 0, 1]
#he was a person
#this is a bag of words

training = []
output = []

#basically what we are doing here is one-hot encoding the data
out_empty = [0 for _ in range(len(labels))]
# print(out_empty) -> [0,0,0,0,0,0]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w.lower()) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    print(output_row)
    training.append(bag)
    output.append(output_row)
    
training = numpy.array(training)
output = numpy.array(output)
print()
print(training)
#the data is now ready to be used to train

[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 0]

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]


In [13]:
#Reset any previous settings
# tensorflow.reset_default_graph()

#input data size
net = tflearn.input_data(shape=[None, len(training[0])])
#hidden layers
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
#output layer, size equal to number of possibilities
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)
#Regular deep neural network
model = tflearn.DNN(net)

print(training.shape)
print(output.shape)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
(26, 46)
(26, 6)


In [14]:
try:
    model.load("chatbot.tflearn")
except:
    model.fit(X_inputs=training, Y_targets=output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("chatbot.tflearn")

Training Step: 3999  | total loss: 0.03929 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.03929 - acc: 0.9993 -- iter: 24/26
Training Step: 4000  | total loss: 0.03855 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.03855 - acc: 0.9993 -- iter: 26/26
--


In [19]:
print("hello")

hello


In [21]:
#convert the input text into bag of words
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
                            
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w==se:
                bag[i] = 1
    
    return numpy.array(bag)


def chat():
    print("Start talking! (quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == 'quit':
            break
        #will hold a list of probabilities (softmax)
        results = model.predict([bag_of_words(inp, words)])
        #will return index of highest probability
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tf['responses']
        
        print(random.choice(responses))
        
chat()

Start talking! (quit to stop)
You: hello
greeting
You: sup
greeting


KeyboardInterrupt: Interrupted by user